# Connection Types

This tutorial focus on the connection types users can utilize from synaptic connections. For better understanding of following connection types, we will combine figure and code for illustration. First import necessary module and initialize a synaptic connection (initialization details please see [Synaptic Connectivity](synaptic_connectivity.ipynb)).

In [7]:
import numpy as np
import brainpy as bp
pre_neuron = np.asarray([0, 0, 0, 1, 1, 1, 1, 2])
post_neuron = np.asarray([3, 5, 7, 0, 2, 4, 6, 1])
conn = bp.conn.IJConn(i=pre_neuron, j=post_neuron)(pre_size=3, post_size=8)

## Matrix-based connection

### Connection matrix `conn_mat`

The matrix-based synaptic connection is one of the most intuitive way to build synaptic computations. The connection matrix between two neuron groups can be easily obtained through the function of connector.requires('conn_mat') (details please see [Synaptic Connectivity](synaptic_connectivity.ipynb)). Each connection matrix is an array with the shape of (num_pre, num_post), like


<img src="../_static/syn-example-conn_mat.png" width="400 px">

In [Synaptic Connectivity](synaptic_connectivity.ipynb), we design two types of synaptic connection: `TwoEndConnector` and `OneEndConnector`. Please notice that in different type of connection, the shape of connection matrix has different meanings.
- `TwoEndConnector`: Because it has two input parameters `pre_size` and `post_size`, each representing the size of pre-synaptic neuron group and post-synaptic neuron group, the shape of the connection matrix is (pre_size, post_size).
- `OneEndConnector`: Because it has only a parameter `pre_size` which represent the size of the neuron group, the shape of the connection matrix is (pre_size, pre_size).

In [8]:
conn.require('conn_mat')

JaxArray(DeviceArray([[False, False, False,  True, False,  True, False,  True],
                      [ True, False,  True, False,  True, False,  True, False],
                      [False,  True, False, False, False, False, False, False]],            dtype=bool))

## Vector-based connection

Matrix-based synaptic computation may be straightforward, but can cause severe wasted RAM memory and inefficient computation. Imaging you want to connect 10,000 pre-synaptic neurons to 10,000 post-synaptic neurons with a 10% random connection probability. Using *matrix*, you need $10^8$ floats to save the synaptic state, and at each update step, you need do computation on $10^8$ floats. Actually, the number of values you really needed is only $10^7$. See, there is a huge memory waste and computing resource inefficiency.

### `pre_ids` and `post_ids`

An effective method to solve this problem is to use *vector* to store the connectivity between neuron groups and the corresponding synaptic states. For the above defined connectivity `conn_mat`, we can align the connected pre-synaptic neurons and the post-synaptic neurons by two one-dimensional arrays: *pre_ids* and *post_ids*,


<img src="../_static/syn-example-pre_ids-post_ids.png" width="700 px">

In such a way, we only need two vectors (`pre_ids` and `post_ids`, each has $10^7$ floats) to store the synaptic connectivity. `syn_id` is also included in the figure, which will be used in next section.

In [9]:
conn.require('pre_ids', 'post_ids')

(JaxArray(DeviceArray([0, 0, 0, 1, 1, 1, 1, 2], dtype=uint32)),
 JaxArray(DeviceArray([3, 5, 7, 0, 2, 4, 6, 1], dtype=uint32)))

### `pre2syn` and `post2syn`

In order to solve the above problem, here we create another two synaptic structures `pre2syn` and `post2syn` to help us retrieve the synapse states which connected with the pre-synaptic neuron $i$ and the post-synaptic neuron $j$.

To better understand `pre2syn` and `post2syn`, the abstract structures are shown in following figures.

In a `pre2syn` list, each `pre2syn[i]` stores the synaptic state indexes projected from the pre-synaptic neuron $i$.

<img src="../_static/syn-example-pre2syn.png" width="300 px">

In a `post2syn` list, each `post2syn[i]` stores the synaptic state indexes projected from the post-synaptic neuron $i$.

<img src="../_static/syn-example-post2syn.png" width="300 px">

However, this data type is not an efficient way to store data, and the data structure in `dict` form cannot be transformed into a `JaxArray`, which is not conducive to subsequent accelerated operations. So here we have optimized the data type by using two vectors for both `pre2syn` and `post2syn`.

For `pre2syn`, the **synapse index vector** records the synaptic state indexes projected from the pre-synaptic neuron in one vector, which means there is no segmentation between each pre-synaptic neuron $i$. The order of **synapse index vector** is determined by the sequence order of **pre-synaptic neuron**.

<img src="../_static/syn-example-pre2syn-1.png" width="700 px">

The **pre index vector** records all the splitting point of **synapse index vector**, constructing a pointer vector for each element points at the beginning index of each pre-synaptic neuron $i$. The last element represent the length of pre-synaptic neuron.

<img src="../_static/syn-example-pre2syn-2.png" width="700 px">

In [10]:
conn.require('pre2syn')

(JaxArray(DeviceArray([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint32)),
 JaxArray(DeviceArray([0, 3, 7, 8], dtype=uint32)))

For `post2syn`, the **synapse index vector** records the synaptic state indexes projected from the post-synaptic neuron in one vector, which means there is no segmentation between each post-synaptic neuron $i$. The order of **synapse index vector** is determined by the sequence order of **post-synpatic neuron**.

<img src="../_static/syn-example-post2syn-1.png" width="700 px">

The **post index vector** records all the splitting point of **synapse index vector**, constructing a pointer vector for each element points at the beginning index of each post-synaptic neuron $i$. The last element represent the length of post-synaptic neuron.

<img src="../_static/syn-example-post2syn-2.png" width="700 px">

In [11]:
conn.require('post2syn')

(JaxArray(DeviceArray([3, 7, 4, 0, 5, 1, 6, 2], dtype=uint32)),
 JaxArray(DeviceArray([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=uint32)))

### `pre2post` and `post2pre`

Another two synaptic structures `pre2post` and `post2pre` establish the direct mapping between the pre-synaptic neurons and the post-synaptic neurons. `pre2post` contains the connected post-synaptic neurons indexes, in which `pre2post[i]` retrieves the post neuron ids projected from pre-synaptic neuron $i$. `post2pre` contains the pre-synaptic neurons indexes, in which `post2pre[j]` retrieves the pre-synaptic neuron ids which project to post-synaptic neuron $j$.

<img src="../_static/syn-example-pre2post.png" width="300 px">

<img src="../_static/syn-example-post2pre.png" width="300 px">

Same as the previous section, we construct two vectors for each connection type for being conducive to subsequent accelerated operations.

For `pre2post`, the **post-synaptic vector** records the post neuron ids projected from pre-synaptic neuron $i$. The order of **post-synaptic vector** is determined by the sequence order of **pre-synaptic neuron**. The **pre index vector** records all the splitting point of **synapse index vector**, constructing a pointer vector for each element points at the beginning index of each pre-synaptic neuron $i$.

<img src="../_static/syn-example-pre2syn-2.png" width="700 px">

In [12]:
conn.require('pre2post')

(JaxArray(DeviceArray([3, 5, 7, 0, 2, 4, 6, 1], dtype=uint32)),
 JaxArray(DeviceArray([0, 3, 7, 8], dtype=uint32)))

For `post2pre`, the **pre-synaptic vector** records the pre neuron ids projected from post-synaptic neuron $i$. The order of **pre-synaptic vector** is determined by the sequence order of **post-synaptic neuron**. The **post index vector** records all the splitting point of **synapse index vector**, constructing a pointer vector for each element points at the beginning index of each post-synaptic neuron $i$.

<img src="../_static/syn-example-post2syn-2.png" width="700 px">

In [13]:
conn.require('post2pre')

(JaxArray(DeviceArray([1, 2, 1, 0, 1, 0, 1, 0], dtype=uint32)),
 JaxArray(DeviceArray([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=uint32)))

---

- Tianqiu Zhang (tianqiuakita@gmail.com)
- Update at 2021.12.12

---